In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import FL_utils.FL_interpretable_functions as interpret
import FL_utils.FL_RiskSLIM as slim
import FL_utils.FL_stumps as stumps
import FL_utils.FL_Corel as Corel

# restore saved variables
%store -r summary_violent2_fl_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\FL Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['violent_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['violent_two_year'].values

In [4]:
#### CART
depth = [1,2,3]
impurity = [0.001, 0.003]
split = [2,3,4]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, split, impurity, seed=816)

### EBM
estimators = [60,80,100]
depth = [1,2]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [8]:
cart_summary['KY_score'], ebm_summary['KY_score']

(0.6161990433587374, 0.7507050306602612)

### Lasso Stumps

In [26]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['violent_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['violent_two_year'].values

## columns 
cols = KY_X.columns

In [10]:
c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [11]:
stumps_summary

{'best_auc': 0.6644129565004342,
 'best_params': {'C': 0.05},
 'auc_diffs': 0.03202071439956211,
 'KY_score': 0.7743573704167719}

### RiskSLIM

In [42]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['violent_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['violent_two_year'].values

## columns 
cols = KY_X.columns

In [43]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.03, random_state=816).fit(FL_X, FL_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

14

In [44]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [45]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'violent_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
10/20/19 @ 11:13 AM | 193 rows in lookup table
10/20/19 @ 11:13 AM | ------------------------------------------------------------
10/20/19 @ 11:13 AM | runnning initialization procedure
10/20/19 @ 11:13 AM | ------------------------------------------------------------
10/20/19 @ 11:13 AM | CPA produced 2 cuts
10/20/19 @ 11:13 AM | running naive rounding on 4 solutions
10/20/19 @ 11:13 AM | best objective value: 0.6931
10/20/19 @ 11:13 AM | rounding produced 4 integer solutions
10/20/19 @ 11:13 AM | best objective value is 0.6931
10/20/19 @ 11:13 AM | running sequential rounding on 4 solutions
10/20/19 @ 11:13 AM | best objective value: 0.6931
10/20/19 @ 11:13 AM | sequential rounding produced 0 integer solutions
10/20/19 @ 11:13 AM | polishing 4 solutions
10/20/19 @ 11:13 AM | best objective value: 0.6931
10/20/19 @ 11:13 AM | polishing produced 4 integer solutions
10/20/19 @ 11:13 AM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 11:13 AM | adding 160 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5218.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 11:15 AM | adding 161 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5067.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 11:16 AM | adding 168 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5107.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 11:18 AM | adding 172 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5009.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

10/20/19 @ 11:20 AM | CPA produced 2 cuts
10/20/19 @ 11:20 AM | running naive rounding on 4 solutions
10/20/19 @ 11:20 AM | best objective value: 0.6931
10/20/19 @ 11:20 AM | rounding produced 4 integer solutions
10/20/19 @ 11:20 AM | best objective value is 0.6931
10/20/19 @ 11:20 AM | running sequential rounding on 4 solutions
10/20/19 @ 11:20 AM | best objective value: 0.6931
10/20/19 @ 11:20 AM | sequential rounding produced 0 integer solutions
10/20/19 @ 11:20 AM | polishing 4 solutions
10/20/19 @ 11:20 AM | best objective value: 0.6931
10/20/19 @ 11:20 AM | polishing produced 4 integer solutions
10/20/19 @ 11:20 AM | initialization produced 7 feasible solutions
10/20/19 @ 11:20 AM | best objective value: 0.5041
10/20/19 @ 11:20 AM | ------------------------------------------------------------
10/20/19 @ 11:20 AM | completed initialization procedure
10/20/19 @ 11:20 AM | ------------------------------------------------------------
10/20/19 @ 11:20 AM | 193 rows in lookup table
CPX

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 11:20 AM | adding 179 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5041.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 1174988 106251        cutoff              0.4977        0.4931  3735090    0.93%           rho_8 U 1174988 1174987     36
 1206648 103517        0.4974     7        0.4977        0.4936  3820670    0.82%           rho_2 N 1206648 705847     34
 1234775 88364        cutoff              0.4977        0.4942  3900952    0.69%           rho_2 D 1234775 107946     29
 1264872 85371        cutoff              0.4977        0.4946  3992993    0.62%           rho_2 N 1264872 824968     29
 1291963 71723        0.4956    11        0.4977        0.4952  4103275    0.50%           rho_4 D 1291963 1191530     29
 1319556 56604        cutoff              0.4977        0.4956  4184452    0.42%           rho_2 U 1319556 1015843     35
 1346820 32969        cutoff              0.4977        0.4961  4251157    0.32%           rho_2 U 1346820 1249643     33
 1381871  3469        cutoff              0.4977        0.4972  4313070    0.09%           rho_3 D 1381871 1267901     40

User cuts applied:  216


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 11:23 AM | adding 165 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5042.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [48]:
riskslim_summary['KY_score'], np.mean(riskslim_summary['FL_train_score'])

(0.7128006541945308, 0.657513058962021)

In [49]:
#### save results
summary_violent2_fl_inter_model = {"CART": cart_summary,
                                    "EBM": ebm_summary,
                                    "Stumps": stumps_summary,
                                    "RiskSLIM": riskslim_summary}
%store summary_violent2_fl_inter_model

Stored 'summary_violent2_fl_inter_model' (dict)


In [50]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Two Year\\"
results = [["Violent",             
            round(cart_summary['KY_score'],3), 
            round(ebm_summary['KY_score'], 3), 
            round(stumps_summary['KY_score'],3), 
            round(riskslim_summary['KY_score'], 3)]]
with open(path + 'Two Year FL_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)